In [26]:
from keras import backend as K
from keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model  # 使用高階模型，keras 無法載入
import numpy as np
import pandas as pd
import os

In [8]:
# 資料路徑
DATASET_PATH  = '/content/drive/My Drive/MyPy/data'
# 影像大小
IMAGE_SIZE = (299, 299) # Xception 預訓練影像大小，resnet 為 (224, 224)
# 影像類別數
NUM_CLASSES = 5
# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 16
# Epoch 數
NUM_EPOCHS = 200
# 模型輸出儲存的檔案
WEIGHTS_BEST = 'model-Xception-best.h5' # 最佳模型
WEIGHTS_LAST = 'model-Xception-last.h5' # 最終模型
# 是否載入已訓練模型
load_mode = True
# 是否使用目前最佳模型
use_best = True

In [9]:
# data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.1)  # set validation split
# 切分訓練和驗證集
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True, 
                                                  subset='training') # set as training data

valid_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True,
                                                  subset='validation') # set as validation data

test_datagen = ImageDataGenerator(rescale=1./255)
test_batches = test_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode=None,
                                                  shuffle=False,
                                                  batch_size=1,)

# 輸出各類別的索引值
print('----- train_batch ----- {} samples'.format(train_batches.samples))
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 輸出各類別的索引值
print('----- valid_batch ----- {} samples'.format(valid_batches.samples))
for cls, idx in valid_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 輸出各類別的索引值
print('----- test_batch ----- {} samples'.format(test_batches.samples))
for cls, idx in test_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 2543 images belonging to 5 classes.
Found 280 images belonging to 5 classes.
Found 2000 images belonging to 1 classes.
----- train_batch ----- 2543 samples
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip
----- valid_batch ----- 280 samples
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip
----- test_batch ----- 2000 samples
Class #0 = all


In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    global load_mode
    
    if load_mode:
        lr = 1e-4
    else:
        lr = 1e-3
    if epoch > 150:
        lr *= 0.5e-3
    elif epoch > 120:
        lr *= 1e-3
    elif epoch > 90:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [27]:
# 准备模型保存路径。
save_dir = os.path.join(DATASET_PATH, 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, WEIGHTS_BEST)

# 准备保存模型和学习速率调整的回调。
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [28]:
# 以訓練好的 model 為基礎來建立模型，
if load_mode :
  # identical to the best one
  net_final = load_model(os.path.join(save_dir, WEIGHTS_BEST))
else:
  # 捨棄 Xception / ResNet50 頂層的 fully connected layers
#  net = ResNet50(include_top=True, weights='imagenet', input_tensor=None,
#                input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))  
  net = Xception(include_top=True, weights='imagenet', input_tensor=None,
                input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
   
  # remove classification layer
  net.layers.pop()  # not work ?
  net = Model(inputs=net.input, outputs=net.layers[-2].output)

  # # 設定凍結與要進行訓練的網路層
  # for layer in net.layers:
  #     layer.trainable = False

  x = net.output
  #x = Flatten()(x)

  # 增加 DropOut layer
  x = Dropout(0.5)(x)

  # 增加 Dense layer，以 softmax 產生個類別的機率值
  output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
  
  net_final = Model(inputs=net.input, outputs=output_layer)

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

Learning rate:  0.0001
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_10[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_____________________________________________________________________

In [29]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        verbose=1,
                        callbacks=callbacks)

# 儲存訓練好的模型
modelpath = os.path.join(save_dir, WEIGHTS_LAST)
net_final.save(modelpath)

Learning rate:  0.0001
Epoch 1/200
158/158 [==============================] - ETA: 0s - loss: 0.0061 - accuracy: 0.9988
Epoch 00001: val_accuracy improved from -inf to 0.94118, saving model to /content/drive/My Drive/MyPy/data/saved_models/model-Xception-best.h5
158/158 [==============================] - 1499s 9s/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 0.2497 - val_accuracy: 0.9412 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 2/200
158/158 [==============================] - ETA: 0s - loss: 0.0037 - accuracy: 0.9984
Epoch 00002: val_accuracy improved from 0.94118 to 0.94853, saving model to /content/drive/My Drive/MyPy/data/saved_models/model-Xception-best.h5
158/158 [==============================] - 105s 663ms/step - loss: 0.0037 - accuracy: 0.9984 - val_loss: 0.1356 - val_accuracy: 0.9485 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 3/200
158/158 [==============================] - ETA: 0s - loss: 0.0037 - accuracy: 0.9992
Epoch 00003: val_accuracy did not improve from 0.

In [13]:
# 載入最佳/最終模型
if use_best :
  net_final = load_model(os.path.join(save_dir, WEIGHTS_BEST))
else:
  net_final = load_model(os.path.join(save_dir, WEIGHTS_LAST))

test_batches.reset()
pred = net_final.predict_generator(test_batches, verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
2000/2000 [==============================] - 1212s 606ms/step


In [14]:
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_batches.class_indices)
label = dict((v,k) for k,v in labels.items())
predictions = [label[i] for i in predicted_class_indices]
# 修改檔名以符合提交格式
filenames = test_batches.filenames
id = [name.replace('.jpg','').replace('all/','') for name in filenames]
# 輸出 csv 檔 id, flower_class
df = pd.DataFrame({"id":id, "flower_class":predicted_class_indices})
df.to_csv(DATASET_PATH + '/submission.csv', index=False)

In [10]:
net_final.evaluate_generator(valid_batches, verbose=1)

18/18 [==============================] - 7s 390ms/step


[0.00455775810405612, 0.8999999761581421]

In [15]:
net_final.evaluate_generator(valid_batches, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
18/18 [==============================] - 155s 9s/step - loss: 0.2075 - accuracy: 0.9357


[0.20748257637023926, 0.9357143044471741]